### Imports

In [48]:
!pip install -q -e ../.

import pandas as pd
import torch
from looptune import prep_config_combinations, single_run, clean_memory
from transformers import Phi3ForSequenceClassification

D:\Users\Jakub\anaconda3\envs\looptune_new_test\Lib\site-packages\IPython\utils\_process_win32.py:124: ResourceWarning: unclosed file <_io.BufferedWriter name=3>
  return process_handler(cmd, _system_body)
D:\Users\Jakub\anaconda3\envs\looptune_new_test\Lib\site-packages\IPython\utils\_process_win32.py:124: ResourceWarning: unclosed file <_io.BufferedReader name=4>
  return process_handler(cmd, _system_body)
D:\Users\Jakub\anaconda3\envs\looptune_new_test\Lib\site-packages\IPython\utils\_process_win32.py:124: ResourceWarning: unclosed file <_io.BufferedReader name=5>
  return process_handler(cmd, _system_body)


### Prepare dataset with two columns: 'text' and 'label'

Examplary data source: https://www.kaggle.com/datasets/ankurzing/sentiment-analysis-for-financial-news


In [53]:
df = pd.read_csv('example_data/SentimentAnalysisforFinancialNews.csv', encoding="ISO-8859-1", header=None)
df.columns = ['label', 'text']
df.head(5)

,label,text
0,neutral,"According to Gran , the company has no plans t..."
1,neutral,Technopolis plans to develop in stages an area...
2,negative,The international electronic industry company ...
3,positive,With the new production plant the company woul...
4,positive,According to the company 's updated strategy f...


### Prepare run configurations

In [67]:
from transformers import Phi3ForSequenceClassification
from transformers import EarlyStoppingCallback

run_config = {   # -----------------------
                  'model_name': [
                            'distilbert/distilbert-base-uncased-finetuned-sst-2-english',
                           #  'michellejieli/emotion_text_classifier',
                           #  'cardiffnlp/twitter-xlm-roberta-base-sentiment',
                           #  'celine98/canine-s-finetuned-sst2',
                           #  'lxyuan/distilbert-base-multilingual-cased-sentiments-student',
                           #  'michelecafagna26/t5-base-finetuned-sst2-sentiment',
                           # 'nlptown/bert-base-multilingual-uncased-sentiment',
                           # 'ProsusAI/finbert',
                           # 'arpanghoshal/EmoRoBERTa',
                           # 'camembert-base'
                           # 'cardiffnlp/twitter-roberta-base-irony',
                           # 'cardiffnlp/twitter-roberta-base-sentiment-latest',
                           # 'ctrl',
                           # 'distilroberta-base',
                           # 'flaubert/flaubert_base_cased',
                           # 'j-hartmann/emotion-english-distilroberta-base',
                           # 'joeddav/distilbert-base-uncased-go-emotions-student',
                           # 'lxyuan/distilbert-base-multilingual-cased-sentiments-student',
                           # 'nlptown/bert-base-multilingual-uncased-sentiment',
                           # 'papluca/xlm-roberta-base-language-detection',
                           # 'roberta-base',
                           # 'xlnet-base-cased',
                           # 'facebook/tart-full-flan-t5-xl',
                           # 'lytang/MiniCheck-Flan-T5-Large',
                          # Need peft on 10GB GPU
                          #  'microsoft/Phi-3-mini-4k-instruct',
                           # 'microsoft/phi-2',
                          # 'meta-llama/Meta-Llama-3-8B',
                           #'lightblue/suzume-llama-3-8B-multilingual',
                           # 'google/gemma-2b',
                           # 'mistralai/Mistral-7B-v0.1',
                           #'mistralai/Mistral-7B-Instruct-v0.1',
                           # 'tiiuae/falcon-11B' ,
                           ], # Pre-trained model names from the Hugging Face hub used for fine-tuning
                    #'custom_loader': Phi3ForSequenceClassification,
                 # --------------------------
                 'split': (0.8, 0.2), # Divides the dataset into training, testing, (and optionally) validation sets. Examples: (0.7,0.3) -> split into train and test proportionally; (70, 30) splits into train,test proportionally.
                 'balanced': (('train',), ('test',)),
                 # --------------------------
                 'training_arguments': {
                     'num_train_epochs': 10, # Number of times the model sees the entire training dataset.
                     'per_device_train_batch_size': 16, # int: Number of samples processed in each training step (personally, 8/16 work best, 16 is faster, but you may find linear drop in inference speed during fine-tuning).
                     'per_device_eval_batch_size': 16, # int:  Number of samples processed in each evaluation step.
                     #'gradient_accumulation_steps': 4,
                     #'gradient_checkpointing': True,
                     #-----------------------------
                     'save_total_limit': 2,
                     'load_best_model_at_end': True,
                     'save_strategy': 'steps', # ('steps', 'epoch' or 'no').
                     'save_steps': 100,
                     'metric_for_best_model': 'eval_loss',
                     #-----------------------------
                     'evaluation_strategy': "steps", # ('steps', 'epoch')
                     'logging_steps': 100, # int
                     #'max_steps': 100, # int
                     'fp16': False, # bool
                     # 'use_cpu': False,
                     #-----------------------------
                     'learning_rate': 5e-5,
                     'lr_scheduler_type': "linear",
                     'warmup_ratio': 0.1,
                     'max_grad_norm': 0.3,
                     'weight_decay': 0.001,
                 },
                 #-----------------------------
                    'trainer': {'callbacks': EarlyStoppingCallback(early_stopping_patience=2)},
                     'bnb_config': [
                                 False,
                                #{'bnb_4bit_compute_dtype': torch.bfloat16, 'load_in_4bit': True, 'bnb_4bit_quant_type': "nf4", 'bnb_4bit_use_double_quant': True, 'load_in_8bit': False}
                                 ],
                 'peft_config': [
                                False,
                                #{'r': 64, 'lora_alpha': 32, 'lora_dropout': 0.1, 'bias': "none",
                                #'task_type': "SEQ_CLS", 
                                #'target_modules': "all-linear",}
                                ],
                    }

run_params_serie = prep_config_combinations(run_config)
print(len(run_params_serie))
run_params_serie


1


[{'model_name': 'distilbert/distilbert-base-uncased-finetuned-sst-2-english',
  'split': (0.8, 0.2),
  'balanced': (('train',), ('test',)),
  'training_arguments': {'num_train_epochs': 10,
   'per_device_train_batch_size': 16,
   'per_device_eval_batch_size': 16,
   'save_total_limit': 2,
   'load_best_model_at_end': True,
   'save_strategy': 'steps',
   'save_steps': 100,
   'metric_for_best_model': 'eval_loss',
   'evaluation_strategy': 'steps',
   'logging_steps': 100,
   'fp16': False,
   'learning_rate': 5e-05,
   'lr_scheduler_type': 'linear',
   'warmup_ratio': 0.1,
   'max_grad_norm': 0.3,
   'weight_decay': 0.001},
  'trainer': {'callbacks': <transformers.trainer_callback.EarlyStoppingCallback at 0x207a12ea350>},
  'bnb_config': False,
  'peft_config': False}]

### Run in loop

In [68]:
for run_params in run_params_serie:
    trainer = single_run(run_params, df, ['trainer'])

          text
label         
negative   604
neutral   2879
positive  1363


Casting to class labels:   0%|          | 0/4846 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/1449 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/363 [00:00<?, ? examples/s]

Map:   0%|          | 0/1449 [00:00<?, ? examples/s]

Map:   0%|          | 0/363 [00:00<?, ? examples/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased-finetuned-sst-2-english and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([2]) in the checkpoint and torch.Size([3]) in the model instantiated
- classifier.weight: found shape torch.Size([2, 768]) in the checkpoint and torch.Size([3, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'callbacks': <transformers.trainer_callback.EarlyStoppingCallback object at 0x00000207A12EA350>}
GPU = NVIDIA GeForce RTX 3080. Max memory = 10.0 GB.
3.506 GB of memory reserved.


Step,Training Loss,Validation Loss,Precision,Recall,F1-score,Accuracy
100,0.760600,0.383754,0.862960,0.862259,0.861653,0.862259
200,0.320600,0.427031,0.874513,0.873278,0.872727,0.873278
300,0.160900,0.710252,0.860433,0.856749,0.854299,0.856749


21.5087 seconds used for training.
0.36 minutes used for training.
Peak reserved memory = 3.506 GB.
Peak reserved memory for training = 0.0 GB.
Peak reserved memory % of max memory = 35.06 %.
Peak reserved memory for training % of max memory = 0.0 %.
